In [1]:
import pandas as pd
import numpy as np

In [2]:
#Import data 

stock_data_2019 = pd.read_excel("fish stock data 2019.xlsx")
stock_data_2021 = pd.read_excel("fish stock data 2020.xlsx")
stock_data_2021 = pd.read_excel("fish stock data 2021.xlsx")
stock_data_2022 = pd.read_excel("fish stock data 2022.xlsx")
stock_data_2023 = pd.read_excel("fish stock data 2023.xlsx")
stock_data_2024 = pd.read_excel("fish stock data 2024.xlsx")

In [3]:
all_stock_data = pd.concat([stock_data_2019, stock_data_2021, stock_data_2022, stock_data_2023, stock_data_2024], ignore_index=True, axis=0)
all_stock_data

,Water name,County,Species,Quantity,Average length,Date stocked
0,ABERDUNK L FL,SEVIER,TIGER TROUT,104,2.99,2019-07-10
1,ABES L W-30,SUMMIT,CUTTHROAT,4021,2.35,2019-09-26
2,ABES RES,SEVIER,BROOK TROUT,108,3.19,2019-07-23
3,ACADEMY MILL RES,SANPETE,TIGER TROUT,3006,2.99,2019-07-10
4,ADAMS RESERVOIR,DAVIS,RAINBOW,500,11.99,2019-03-18
...,...,...,...,...,...,...
12176,YUBA RES (SEVIER BRG,JUAB,CHANNEL CATFISH,470,13.57,2024-06-20
12177,YUBA RES (SEVIER BRG,JUAB,MUSKIE TIGER,2121,6.42,2024-07-08
12178,YUBA RES (SEVIER BRG,JUAB,WIPER,50220,2.04,2024-07-18
12179,YUBA RES (SEVIER BRG,JUAB,RAINBOW,3332,10.27,2024-10-30
